***Connect to google drive colab***

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


***Installation of dependencies***

In [ ]:
!pip install assemblyai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.6/72.6 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.1/164.1 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.3 MB/s eta 0:00:00


In [ ]:
!pip install deep_translator

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 1.9 MB/s eta 0:00:00


***Display audio***

In [ ]:
from IPython.display import Audio

# Function to handle audio playback
def play_audio(file_path, rate=None):
    """Plays an audio file.

    Args:
        file_path (str): The path to the audio file.
        rate (int, optional): The sample rate of the audio.
                               If None, attempts to autodetect.
                               Defaults to None.
    """
    # If rate is not provided, assume it's an audio file and let Audio autodetect
    # If rate is provided, assume data is numpy array and pass rate
    audio = Audio(filename=file_path, rate=44100)
    display(audio)

audio_file = "/content/drive/MyDrive/AI_ML_INTERN /Task 3 Practice /vitals3.mp3"
play_audio(audio_file)

Output hidden; open in https://colab.research.google.com to view.

***Speech recognation***

In [ ]:
import requests
import json
import time

def read_file(filename, chunk_size=5242880):
    with open(filename, 'rb') as _file:
        while True:
            data = _file.read(chunk_size)
            if not data:
                break
            yield data

def upload_file(api_token, path):
    """
    Upload a file to the AssemblyAI API.

    Args:
        api_token (str): Your API token for AssemblyAI.
        path (str): Path to the local file.

    Returns:
        str: The upload URL.
    """
    headers = {'authorization': api_token}
    response = requests.post('https://api.assemblyai.com/v2/upload',
                             headers=headers,
                             data=read_file(path))

    if response.status_code == 200:
        return response.json()["upload_url"]
    else:
        print(f"Error: {response.status_code} - {response.text}")
        return None

def create_transcript(api_token, audio_url):
    """
    Create a transcript using AssemblyAI API.

    Args:
        api_token (str): Your API token for AssemblyAI.
        audio_url (str): URL of the audio file to be transcribed.

    Returns:
        dict: Completed transcript object.
    """
    # Set the API endpoint for creating a new transcript
    url = "https://api.assemblyai.com/v2/transcript"

    # Set the headers for the request, including the API token and content type
    headers = {
        "authorization": api_token,
        "content-type": "application/json"
    }

    # Set the data for the request, including the URL of the audio file to be transcribed
    data = {
        "audio_url": audio_url
    }

    # Send a POST request to the API to create a new transcript, passing in the headers and data
    response = requests.post(url, json=data, headers=headers)

    # Get the transcript ID from the response JSON data
    transcript_id = response.json()['id']

    # Set the polling endpoint URL by appending the transcript ID to the API endpoint
    polling_endpoint = f"https://api.assemblyai.com/v2/transcript/{transcript_id}"

    # Keep polling the API until the transcription is complete
    while True:
        # Send a GET request to the polling endpoint, passing in the headers
        transcription_result = requests.get(polling_endpoint, headers=headers).json()

        # If the status of the transcription is 'completed', exit the loop
        if transcription_result['status'] == 'completed':
            break

        # If the status of the transcription is 'error', raise a runtime error with the error message
        elif transcription_result['status'] == 'error':
            raise RuntimeError(f"Transcription failed: {transcription_result['error']}")

        # If the status of the transcription is not 'completed' or 'error', wait for 3 seconds and poll again
        else:
            time.sleep(3)

    return transcription_result

your_api_token = "bc7e213c3d14435d9f1e198ce37badb2"

# Upload a local file
filename = "/content/drive/MyDrive/AI_ML_INTERN /Task 3 Practice /vitals3.mp3"
upload_url = upload_file(your_api_token, filename)

# Transcribe it
transcript = create_transcript(your_api_token, upload_url)

# Print the completed transcript object
result=transcript['text']
print(result)

Good morning, Mr. Williams. I'll be doing a full set of advanced vitals today. Are you comfortable? Yes, I'm ready. Thanks for checking. Alright, let's begin. First, I'll take your blood pressure. Just relax your arm here, please. Okay. Your blood pressure is 124 over 78, which is in the normal range. Now I'll move your heart rate variability. I am placing these sensors on your chest to monitor your heart rhythm. Sure, no problem. Your heart rate is. Heart rate variability is good, indicating that your heart is responding well to changes. Now I'll measure your respiratory rate. Just breathe normally. Alright. Your respiratory rate is.16 breaths per minute is perfectly within range. Now let me check your blood oxygen level. I'll clip this device on your finger. OK. Your oxygen saturation is 99%, which is great. Finally, I'll use this thermometer to get your core temperature. Your temperature is 98.7 degrees Fahrenheit. So that's normal. All of your vitals look stable today, Mr. Williams

In [ ]:
print(transcript)

{'id': '44d717d7-06e3-45db-8aae-8bf4be728315', 'language_model': 'assemblyai_default', 'acoustic_model': 'assemblyai_default', 'language_code': 'en_us', 'status': 'completed', 'audio_url': 'https://cdn.assemblyai.com/upload/1cd29a0d-76a3-4d6f-88bf-1faddb5ddf50', 'text': "Good morning, Mr. Williams. I'll be doing a full set of advanced vitals today. Are you comfortable? Yes, I'm ready. Thanks for checking. Alright, let's begin. First, I'll take your blood pressure. Just relax your arm here, please. Okay. Your blood pressure is 124 over 78, which is in the normal range. Now I'll move your heart rate variability. I am placing these sensors on your chest to monitor your heart rhythm. Sure, no problem. Your heart rate is. Heart rate variability is good, indicating that your heart is responding well to changes. Now I'll measure your respiratory rate. Just breathe normally. Alright. Your respiratory rate is.16 breaths per minute is perfectly within range. Now let me check your blood oxygen le

'audio_url': 'https://cdn.assemblyai.com/upload/35c6cb08-b9bd-481e-9682-3cbbb0f67397'

In [ ]:
# Split the transcript into chunks of maximum length 80 characters
chunks = [transcript['text'][i:i+110] for i in range(0, len(transcript['text']), 110)]

# Join the chunks with line breaks
formatted_text = "\n".join(chunks)

print(formatted_text)

Good morning, Mr. Williams. I'll be doing a full set of advanced vitals today. Are you comfortable? Yes, I'm r
eady. Thanks for checking. Alright, let's begin. First, I'll take your blood pressure. Just relax your arm her
e, please. Okay. Your blood pressure is 124 over 78, which is in the normal range. Now I'll move your heart ra
te variability. I am placing these sensors on your chest to monitor your heart rhythm. Sure, no problem. Your 
heart rate is. Heart rate variability is good, indicating that your heart is responding well to changes. Now I
'll measure your respiratory rate. Just breathe normally. Alright. Your respiratory rate is.16 breaths per min
ute is perfectly within range. Now let me check your blood oxygen level. I'll clip this device on your finger.
 OK. Your oxygen saturation is 99%, which is great. Finally, I'll use this thermometer to get your core temper
ature. Your temperature is 98.7 degrees Fahrenheit. So that's normal. All of your vitals look stable today, Mr
.

***Speaker Diarization***

In [ ]:
import requests
import json
import time

def read_file(filename, chunk_size=5242880):
    with open(filename, 'rb') as _file:
        while True:
            data = _file.read(chunk_size)
            if not data:
                break
            yield data

def upload_file(api_token, path):
    """
    Upload a file to the AssemblyAI API.

    Args:
        api_token (str): Your API token for AssemblyAI.
        path (str): Path to the local file.

    Returns:
        str: The upload URL.
    """
    headers = {'authorization': api_token}
    response = requests.post('https://api.assemblyai.com/v2/upload',
                             headers=headers,
                             data=read_file(path))

    if response.status_code == 200:
        return response.json()["upload_url"]
    else:
        print(f"Error: {response.status_code} - {response.text}")
        return None

def create_transcript(api_token, audio_url):
    """
    Create a transcript using AssemblyAI API.

    Args:
        api_token (str): Your API token for AssemblyAI.
        audio_url (str): URL of the audio file to be transcribed.

    Returns:
        dict: Completed transcript object.
    """
    # Set the API endpoint for creating a new transcript
    url = "https://api.assemblyai.com/v2/transcript"

    # Set the headers for the request, including the API token and content type
    headers = {
        "authorization": api_token,
        "content-type": "application/json"
    }

    # Set the data for the request, including the URL of the audio file to be transcribed
    data = {
        "audio_url": audio_url,
        "speaker_labels": True
    }

    # Send a POST request to the API to create a new transcript, passing in the headers and data
    response = requests.post(url, json=data, headers=headers)

    # Get the transcript ID from the response JSON data
    transcript_id = response.json()['id']

    # Set the polling endpoint URL by appending the transcript ID to the API endpoint
    polling_endpoint = f"https://api.assemblyai.com/v2/transcript/{transcript_id}"

    # Keep polling the API until the transcription is complete
    while True:
        # Send a GET request to the polling endpoint, passing in the headers
        transcription_result = requests.get(polling_endpoint, headers=headers).json()

        # If the status of the transcription is 'completed', exit the loop
        if transcription_result['status'] == 'completed':
            break

        # If the status of the transcription is 'error', raise a runtime error with the error message
        elif transcription_result['status'] == 'error':
            raise RuntimeError(f"Transcription failed: {transcription_result['error']}")

        # If the status of the transcription is not 'completed' or 'error', wait for 3 seconds and poll again
        else:
            time.sleep(3)

    return transcription_result

your_api_token = "bc7e213c3d14435d9f1e198ce37badb2"

# Upload a local file
filename = "/content/drive/MyDrive/AI_ML_INTERN /Task 3 Practice /vitals3.mp3"
upload_url = upload_file(your_api_token, filename)

# Transcribe it
transcrSpeaker = create_transcript(your_api_token, upload_url)

# Iterate through the utterances and print speaker information
print("\nSpeakers' Information:\n\n")

for utterance in transcrSpeaker['utterances']:
    speaker = utterance['speaker']
    text = utterance['text']
    confidence = utterance['confidence']

    print(f"Speaker {speaker}: {text}\nConfidence: {confidence}\n")


Speakers' Information:


Speaker A: Good morning, Mr. Williams. I'll be doing a full set of advanced vitals today. Are you comfortable? Yes, I'm ready. Thanks for checking. Alright, let's begin. First, I'll take your blood pressure. Just relax your arm here, please. Okay. Your blood pressure is 124 over 78, which is in the normal range. Now I'll move your heart rate variability. I am placing these sensors on your chest to monitor your heart rhythm. Sure, no problem. Your heart rate is. Heart rate variability is good, indicating that your heart is responding well to changes. Now I'll measure your respiratory rate. Just breathe normally. Alright. Your respiratory rate is.16 breaths per minute is perfectly within range. Now let me check your blood oxygen level. I'll clip this device on your finger. OK. Your oxygen saturation is 99%, which is great. Finally, I'll use this thermometer to get your core temperature. Your temperature is 98.7 degrees Fahrenheit. So that's normal. All of your v

In [ ]:
print(transcrSpeaker['utterances'])

[{'speaker': 'A', 'text': "Good morning, Mr. Williams. I'll be doing a full set of advanced vitals today. Are you comfortable? Yes, I'm ready. Thanks for checking. Alright, let's begin. First, I'll take your blood pressure. Just relax your arm here, please. Okay. Your blood pressure is 124 over 78, which is in the normal range. Now I'll move your heart rate variability. I am placing these sensors on your chest to monitor your heart rhythm. Sure, no problem. Your heart rate is. Heart rate variability is good, indicating that your heart is responding well to changes. Now I'll measure your respiratory rate. Just breathe normally. Alright. Your respiratory rate is.16 breaths per minute is perfectly within range. Now let me check your blood oxygen level. I'll clip this device on your finger. OK. Your oxygen saturation is 99%, which is great. Finally, I'll use this thermometer to get your core temperature. Your temperature is 98.7 degrees Fahrenheit. So that's normal. All of your vitals look

***Summarization***

In [ ]:
import requests
import json
import time

def read_file(filename, chunk_size=5242880):
    with open(filename, 'rb') as _file:
        while True:
            data = _file.read(chunk_size)
            if not data:
                break
            yield data

def upload_file(api_token, path):
    """
    Upload a file to the AssemblyAI API.

    Args:
        api_token (str): Your API token for AssemblyAI.
        path (str): Path to the local file.

    Returns:
        str: The upload URL.
    """
    headers = {'authorization': api_token}
    response = requests.post('https://api.assemblyai.com/v2/upload',
                             headers=headers,
                             data=read_file(path))

    if response.status_code == 200:
        return response.json()["upload_url"]
    else:
        print(f"Error: {response.status_code} - {response.text}")
        return None

def create_transcript(api_token, audio_url):
    """
    Create a transcript using AssemblyAI API.

    Args:
        api_token (str): Your API token for AssemblyAI.
        audio_url (str): URL of the audio file to be transcribed.

    Returns:
        dict: Completed transcript object.
    """
    # Set the API endpoint for creating a new transcript
    url = "https://api.assemblyai.com/v2/transcript"

    # Set the headers for the request, including the API token and content type
    headers = {
        "authorization": api_token,
        "content-type": "application/json"
    }

    # Set the data for the request, including the URL of the audio file to be transcribed
    data = {
        "audio_url": audio_url,
        "summarization": True,
        "summary_model": "informative",
        "summary_type": "bullets"
    }

    # Send a POST request to the API to create a new transcript, passing in the headers and data
    response = requests.post(url, json=data, headers=headers)

    # Get the transcript ID from the response JSON data
    transcript_id = response.json()['id']

    # Set the polling endpoint URL by appending the transcript ID to the API endpoint
    polling_endpoint = f"https://api.assemblyai.com/v2/transcript/{transcript_id}"

    # Keep polling the API until the transcription is complete
    while True:
        # Send a GET request to the polling endpoint, passing in the headers
        transcription_result = requests.get(polling_endpoint, headers=headers).json()

        # If the status of the transcription is 'completed', exit the loop
        if transcription_result['status'] == 'completed':
            break

        # If the status of the transcription is 'error', raise a runtime error with the error message
        elif transcription_result['status'] == 'error':
            raise RuntimeError(f"Transcription failed: {transcription_result['error']}")

        # If the status of the transcription is not 'completed' or 'error', wait for 3 seconds and poll again
        else:
            time.sleep(3)

    return transcription_result

your_api_token = "bc7e213c3d14435d9f1e198ce37badb2"

# Upload a local file
filename = "/content/drive/MyDrive/AI_ML_INTERN /Task 3 Practice /vitals3.mp3"
upload_url = upload_file(your_api_token, filename)

# Transcribe it
transcript = create_transcript(your_api_token, upload_url)

# Print the summary
print(transcript["summary"])

- Mr. Williams will undergo a full set of advanced vitals today. All of your vitals look stable today, Mr. Williams. If you are experiencing any shortness of breath or other symptoms, let us know right away.


***Translate the text from english to any language the user want***

In [ ]:
from deep_translator import GoogleTranslator
translator = GoogleTranslator(source='auto', target='fr')
# Split the text into smaller chunks
chunk_size = 500  # Adjust as per your needs
chunks = [result[i:i+chunk_size] for i in range(0, len(result), chunk_size)]
# Translate each chunk and store the results
translations = []
for chunk in chunks:
    translation = translator.translate(chunk)
    translations.append(translation)
# Concatenate the translations
translated_text = " ".join(translations)
print(translated_text)

Bonjour, M. Williams. Je vais faire une série complète de signes vitaux avancés aujourd'hui. Êtes-vous à l'aise ? Oui, je suis prêt. Merci d'avoir vérifié. Très bien, commençons. Tout d'abord, je vais prendre votre tension artérielle. Détendez simplement votre bras ici, s'il vous plaît. OK. Votre tension artérielle est de 124 sur 78, ce qui est dans la plage normale. Maintenant, je vais déplacer la variabilité de votre fréquence cardiaque. Je place ces capteurs sur votre poitrine pour surveiller votre rythme cardiaque. Bien sûr, pas de problème. Votre fréquence cardiaque est. La variabilité de la fréquence cardiaque est bonne, ce qui indique que Votre cœur réagit bien aux changements. Je vais maintenant mesurer votre fréquence respiratoire. Respirez normalement. Très bien. Votre fréquence respiratoire est de 0,16 respirations par minute, ce qui est tout à fait dans les limites. Maintenant, laissez-moi vérifier votre taux d'oxygène dans le sang. Je vais attacher cet appareil à votre doi

In [ ]:
from deep_translator import GoogleTranslator

translator = GoogleTranslator(source='auto', target='de')


# Split the text into smaller chunks
chunk_size = 500  # Adjust as per your needs
chunks = [transcript["summary"][i:i+chunk_size] for i in range(0, len(transcript["summary"]), chunk_size)]

# Translate each chunk and store the results
translations = []
for chunk in chunks:
    translation = translator.translate(chunk)
    translations.append(translation)

# Concatenate the translations
translated_text = " ".join(translations)

print(translated_text)



- Herr Williams wird heute einer vollständigen Reihe von erweiterten Vitalfunktionen unterzogen. Alle Ihre Vitalfunktionen scheinen heute stabil zu sein, Herr Williams. Wenn Sie Kurzatmigkeit oder andere Symptome verspüren, informieren Sie uns sofort.


In [ ]:
# Split the transcript into chunks of maximum length 80 characters
chunks = [translated_text[i:i+110] for i in range(0, len(translated_text), 110)]

# Join the chunks with line breaks
formatted_text_Trans= "\n".join(chunks)

print(formatted_text_Trans)

- Herr Williams wird heute einer vollständigen Reihe von erweiterten Vitalfunktionen unterzogen. Alle Ihre Vit
alfunktionen scheinen heute stabil zu sein, Herr Williams. Wenn Sie Kurzatmigkeit oder andere Symptome verspür
en, informieren Sie uns sofort.


In [ ]:
from deep_translator import GoogleTranslator

translator = GoogleTranslator(source='auto', target='mr')

# Iterate through the utterances and print translated speaker information
print("\nInformation des intervenants:\n\n")

for utterance in transcrSpeaker['utterances']:
    speaker = utterance['speaker']
    text = utterance['text']
    confidence = utterance['confidence']

    # Translate the speaker and text
    translated_speaker = translator.translate(speaker)
    translated_text = translator.translate(text)

    print(f"Intervenant {translated_speaker}: {translated_text}\nConfiance: {confidence}\n")



Information des intervenants:


Intervenant ए: सुप्रभात, मिस्टर विल्यम्स. मी आज प्रगत जीवनावश्यक गोष्टींचा संपूर्ण संच करणार आहे. आपण आरामदायक आहात? होय, मी तयार आहे. तपासल्याबद्दल धन्यवाद. ठीक आहे, चला सुरुवात करूया. प्रथम, मी तुमचा रक्तदाब घेईन. कृपया इथे तुमचा हात आराम करा. ठीक आहे. तुमचा रक्तदाब १२४ पेक्षा ७८ आहे, जो सामान्य श्रेणीत आहे. आता मी तुमची हृदय गती बदलते. तुमच्या हृदयाच्या तालावर लक्ष ठेवण्यासाठी मी हे सेन्सर्स तुमच्या छातीवर ठेवत आहे. नक्कीच, काही हरकत नाही. तुमचे हृदय गती आहे. हृदय गती परिवर्तनशीलता चांगली आहे, हे दर्शविते की तुमचे हृदय बदलांना चांगला प्रतिसाद देत आहे. आता मी तुमचा श्वसनाचा वेग मोजतो. फक्त सामान्यपणे श्वास घ्या. ठीक आहे. तुमचा श्वासोच्छ्वास दर मिनिटाला १६ श्वासोच्छ्वासाच्या मर्यादेत आहे. आता मला तुमची रक्तातील ऑक्सिजन पातळी तपासू द्या. मी हे उपकरण तुमच्या बोटावर क्लिप करेन. ठीक आहे. तुमचे ऑक्सिजन संपृक्तता 99% आहे, जे उत्तम आहे. शेवटी, तुमचे कोर तापमान जाणून घेण्यासाठी मी हे थर्मामीटर वापरेन. तुमचे तापमान ९८.७ अंश फॅरेनहाइट आहे. तर ते सामान्य आहे. मिस